# Code Specialist 7B

Code Specialist 7B es un modelo afinado sobre `mistralai/Mistral-7B-Instruct-v0.3` con foco en generación y explicación de código en Python y SQL para tareas de analitica y ciencia de datos.


### Test de compatibilidad de versiones
importar librerÍas, imprimir sus versiones y confirmar si `torch.cuda.is_available()`

In [ ]:
import torch
import transformers
import accelerate
import os
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# Suprimir advertencia de bitsandbytes
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

print("Librerías importadas con éxito.")

print("--- Verificación de Versiones de Librerías ---")
print(f"PyTorch:         {torch.__version__}")
print(f"CUDA (PyTorch):  {torch.version.cuda}")
print(f"Transformers:    {transformers.__version__}")
print(f"Datasets:        {datasets.__version__}")
print(f"PEFT:            {peft.__version__}")
print(f"TRL:             {trl.__version__}")
print(f"BitsAndBytes:    {bitsandbytes.__version__}")
print(f"Accelerate:      {accelerate.__version__}")
print("-----------------------------------------------")

# Verificación final de disponibilidad de CUDA
print(f"\nCUDA disponible para PyTorch: {torch.cuda.is_available()}")

--- Verificación de Versiones de Librerías ---
PyTorch:         2.6.0+cu124
CUDA (PyTorch):  12.4
Transformers:    4.56.2
Datasets:        4.1.1
PEFT:            0.17.1
TRL:             0.23.0
BitsAndBytes:    0.47.0
Accelerate:      1.10.1
-----------------------------------------------

CUDA disponible para PyTorch: True


### Cargar modelo Mistral

Descarga el checkpoint base indicado en `MODEL_ID`, inicializa el tokenizador y aplica cuantización 4-bit mediante `BitsAndBytesConfig` para reducir el consumo de VRAM durante el fine-tuning.

**Configuración clave**
- `load_in_4bit=True` y `bnb_4bit_compute_dtype=torch.bfloat16` habilitan entrenamiento eficiente conservando estabilidad numérica.
- El tokenizador usa `pad_token = eos_token` para evitar inconsistencias con secuencias empaquetadas.
- `LoraConfig` limita el entrenamiento a los módulos de atención (`q_proj`, `k_proj`, `v_proj`, `o_proj`) logrando especialización en código con pocos millones de parametros adicionales.

In [ ]:
# Definir el Modelo y el Tokenizador
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"
NUEVO_MODELO_NOMBRE = "Mistral-7B-Python-Expert-v1"

print(f"Cargando el modelo base: {MODEL_ID}")

# Configuración de Cuantización (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Cargar el Tokenizador 
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizador cargado.")

# Cargar el Modelo Base con Cuantización
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto" # "auto" le dice a transformers que ponga el modelo en la GPU
)

print(f"Modelo {MODEL_ID} cargado en 4-bits.")

# Configurar LoRA (PEFT)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Aplica LoRA al Modelo 
model = get_peft_model(model, lora_config)

print("Configuración de LoRA (PEFT) aplicada al modelo.")
model.print_trainable_parameters()

Librerías importadas con éxito.
Cargando el modelo base: mistralai/Mistral-7B-Instruct-v0.3
Tokenizador cargado.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Modelo mistralai/Mistral-7B-Instruct-v0.3 cargado en 4-bits.
Configuración de LoRA (PEFT) aplicada al modelo.
trainable params: 13,631,488 || all params: 7,261,655,040 || trainable%: 0.1877


### Cargar y Procesar Datasets

El ajuste fino usa dos corpus abiertos en formato Alpaca: `sahil2801/CodeAlpaca-20k` y `TokenBender/code_instructions_122k_alpaca_style`. Para especializarnos en Python/SQL:
1. Definimos `PYTHON_KEYWORDS_SIMPLE` con terminología de ciencia de datos.
2. `es_python_simple` concatena `instruction` + `output` y conserva ejemplos que contengan alguna keyword.
3. Cada ejemplo filtrado se transforma a la plantilla `[INST] ... [/INST] respuesta</s>` mediante `format_prompt_mistral_alpaca`, descartando los que no tienen `output`.
4. Se combinan los datasets, se mezclan con `seed=42` y se muestran ejemplos de verificaci?n para inspeccionar manualmente el formato final.

In [ ]:
from datasets import load_dataset, concatenate_datasets
import re

print("Iniciando procesamiento (Solo Datasets de Código)")

# Filtrar ejemplos de Python y SQL
PYTHON_KEYWORDS_SIMPLE = set(["python", "pandas", "numpy", "def ", "sklearn", "torch", "tensorflow","sql"])

def es_python_simple(example):
    texto_completo = (example['instruction'] + example['output']).lower()
    return any(keyword in texto_completo for keyword in PYTHON_KEYWORDS_SIMPLE)

# Cargar y Filtrar CodeAlpaca
print("Cargando CodeAlpaca...")
ds_alpaca = load_dataset("sahil2801/CodeAlpaca-20k", split="train")
ds_alpaca_python = ds_alpaca.filter(es_python_simple)
print(f"CodeAlpaca: {len(ds_alpaca)} originales -> {len(ds_alpaca_python)} de Python")

# Cargar y Filtrar CodeInstructions122k
print("Cargando CodeInstructions122k...")
ds_code_122k = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")
ds_code_122k_python = ds_code_122k.filter(es_python_simple)
print(f"CodeInstructions122k: {len(ds_code_122k)} originales -> {len(ds_code_122k_python)} de Python")

# Formatear para SFT
# Ambos datasets usan el formato 'instruction', 'input', 'output' de Alpaca
def format_prompt_mistral_alpaca(example):
    instruccion = example['instruction']
    input_opcional = example.get('input') # .get() no da error si 'input' no existe
    respuesta = example['output']

    # Si hay un 'input' (contexto adicional), lo unimos a la instrucción
    if input_opcional:
        instruccion = instruccion + "\n\nInput:\n" + input_opcional
    
    # Descartar ejemplos sin respuesta
    if not respuesta:
        return None

    example['text'] = f"[INST] {instruccion} [/INST] {respuesta}</s>"
    return example

print("Aplicando formato Mistral a los datasets...")
ds_alpaca_formatted = ds_alpaca_python.map(format_prompt_mistral_alpaca, remove_columns=['instruction', 'input', 'output'])

ds_code_122k_formatted = ds_code_122k_python.map(format_prompt_mistral_alpaca, remove_columns=['instruction', 'input', 'output']) # <-- ¡CORREGIDO!

# Filtrar cualquier ejemplo que haya devuelto None
ds_alpaca_formatted = ds_alpaca_formatted.filter(lambda x: x['text'] is not None)
ds_code_122k_formatted = ds_code_122k_formatted.filter(lambda x: x['text'] is not None)

#  Combinar y Mezclar 
dataset_final = concatenate_datasets([ds_alpaca_formatted, ds_code_122k_formatted])
dataset_final = dataset_final.shuffle(seed=42)

print("\n--- ¡Procesamiento completado! ---")
print(f"Tamaño total del dataset final: {len(dataset_final)}")

# Imprimir un ejemplo para verificar el formato
print("\n--- Ejemplo de formato (índice 0): ---")
print(dataset_final[0]['text'])
print("\n--- Ejemplo de formato (índice 500): ---")
print(dataset_final[500]['text'])

Iniciando procesamiento (Solo Datasets de Código)
Cargando CodeAlpaca...
CodeAlpaca: 20022 originales -> 8578 de Python
Cargando CodeInstructions122k...
CodeInstructions122k: 121959 originales -> 63841 de Python
Aplicando formato Mistral a los datasets...

--- ¡Procesamiento completado! ---
Tamaño total del dataset final: 72418

--- Ejemplo de formato (índice 0): ---
[INST] Create a Python function that takes a dictionary as an argument, and returns the value with maximum frequency in that dictionary.

Input:
dic = {1: 'a', 2: 'b', 3: 'b', 4: 'c'} [/INST] def max_freq_val(dic): 
    max_freq = 0
    max_val = None
    for val in dic.values():
        if dic.values().count(val) > max_freq:
            max_freq = dic.values().count(val)
            max_val = val
    
    return max_val

max_val = max_freq_val(dic)
print(max_val)</s>

--- Ejemplo de formato (índice 500): ---
[INST] Create an algorithm to merge two sorted linked lists into one sorted list. [/INST] def merge_sorted_lists(ls

### Entrenamiento

Con el dataset final se realiza un split 95/5 para evaluación y se configura `SFTConfig`/`SFTTrainer`:
- La precisión es dinamica: BF16 si la GPU lo soporta; de lo contrario FP16.
- `per_device_train_batch_size=2` + `gradient_accumulation_steps=4` emula un batch efectivo de 8 ejemplos.
- Estrategias de logging/guardado (`logging_steps`, `save_steps`, `save_total_limit`) controlan la frecuencia de checkpoints y el uso de disco.
- Se detecta automáticamente el parámetro correcto (`evaluation_strategy` vs `eval_strategy`)
- Tras el entrenamiento se evalúa, se calcula la perplexity y se escriben los adaptadores LoRA.

Se puede reanudar un entrenamiento interrumpido si existen checkpoints previos en la carpeta de salida. Para trabajos mas largos ajusta `num_train_epochs`, `max_length` o `packing` según la longitud de los prompts.


In [ ]:
import os, re, math, torch, inspect
from datasets import DatasetDict
from trl import SFTTrainer, SFTConfig

# Split train/eval 
splits = dataset_final.train_test_split(test_size=0.05, seed=42)  # 5% eval
train_ds = splits["train"]
eval_ds  = splits["test"]
print(f"[Split] Train: {len(train_ds)} | Eval: {len(eval_ds)}")

# Precisión coherente con GPU
use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
precision_kwargs = dict(bf16=True, fp16=False) if use_bf16 else dict(bf16=False, fp16=True)
print(f"[Precision] bf16={precision_kwargs.get('bf16')} fp16={precision_kwargs.get('fp16')}")

# Hiperparámetros 
PER_DEVICE_BS = 2
GRAD_ACCUM    = 4
PACKING       = False
MAX_LEN       = 1024
LOG_STEPS     = 50
SAVE_STEPS    = 1000
EVAL_STEPS    = SAVE_STEPS
SAVE_LIMIT    = 2

# Detectar nombre correcto del parámetro de evaluación
sft_params = set(inspect.signature(SFTConfig.__init__).parameters.keys())
EVAL_KEY = "evaluation_strategy" if "evaluation_strategy" in sft_params else (
           "eval_strategy"        if "eval_strategy"        in sft_params else None)

if EVAL_KEY is None:
    print("[Aviso] Ni 'evaluation_strategy' ni 'eval_strategy' están en SFTConfig; se hará evaluación solo al final.")
else:
    print(f"[Eval key] Usando '{EVAL_KEY}' para estrategia de evaluación.")

# Construir kwargs comunes de SFTConfig
base_kwargs = dict(
    output_dir=NUEVO_MODELO_NOMBRE,
    per_device_train_batch_size=PER_DEVICE_BS,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=SAVE_LIMIT,
    dataset_text_field="text",
    max_length=MAX_LEN,
    packing=PACKING,
    gradient_checkpointing=True,
    report_to="none",
    pad_token=tokenizer.pad_token,
    eos_token=tokenizer.eos_token,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    seed=42,
    **precision_kwargs
)

if EVAL_KEY:
    base_kwargs[EVAL_KEY] = "steps"
    base_kwargs["eval_steps"] = EVAL_STEPS

# Instanciar SFTConfig 
sft_config = SFTConfig(**base_kwargs)

#  Crear SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    args=sft_config,
)

# Reanuda desde último checkpoint si existe
last_ckpt = None
if os.path.isdir(NUEVO_MODELO_NOMBRE):
    cks = [d for d in os.listdir(NUEVO_MODELO_NOMBRE) if d.startswith("checkpoint-")]
    if cks:
        steps = []
        for c in cks:
            m = re.findall(r"checkpoint-(\d+)", c)
            if m:
                steps.append(int(m[0]))
        if steps:
            last_ckpt = os.path.join(NUEVO_MODELO_NOMBRE, f"checkpoint-{max(steps)}")
            print(f"[Resume] Reanudando desde: {last_ckpt}")

#  Entrenamiento
print("[Train] ¡Iniciando entrenamiento!")
trainer.train(resume_from_checkpoint=last_ckpt)

# Evaluación final + Perplexity
print("[Eval] Evaluación final...")
metrics = trainer.evaluate()
for k, v in metrics.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.6f}")
    else:
        print(f"  {k}: {v}")

if "eval_loss" in metrics and isinstance(metrics["eval_loss"], float):
    try:
        ppl = math.exp(metrics["eval_loss"])
        print(f"  Perplexity: {ppl:.2f}")
    except OverflowError:
        pass

# Guardar adaptadores finales LoRA
final_dir = f"{NUEVO_MODELO_NOMBRE}-final"
trainer.save_model(final_dir)
print(f"[Save] Adaptadores LoRA guardados en: {final_dir}")


[Split] Train: 68797 | Eval: 3621
[Precision] bf16=True fp16=False
[Eval key] Usando 'eval_strategy' para estrategia de evaluación.
[Resume] Reanudando desde: Mistral-7B-Python-Expert-v1\checkpoint-7000
[Train] ¡Iniciando entrenamiento!


c:\Users\fabri\anaconda3\envs\nlp311\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
8000,1.798600,1.751899,0.495269,1301355.000000,0.747835


c:\Users\fabri\anaconda3\envs\nlp311\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[Eval] Evaluación final...


  eval_loss: 1.751899
  eval_runtime: 1181.960000
  eval_samples_per_second: 3.064000
  eval_steps_per_second: 0.383000
  eval_entropy: 0.495269
  eval_num_tokens: 2077547.000000
  eval_mean_token_accuracy: 0.747835
  epoch: 0.999913
  Perplexity: 5.77
[Save] Adaptadores LoRA guardados en: Mistral-7B-Python-Expert-v1-final


### Merge del modelo

Se convierten los adaptadores LoRA en un checkpoint estandar de Transformers:
1. Se limpian recursos en GPU, se cargan nuevamente el modelo base y los adaptadores (`PeftModel`).
2. `merge_and_unload()` aplica las actualizaciones LoRA directamente sobre los pesos del modelo base.
3. Se guardan los pesos fusionados y el tokenizador en `MERGED_MODEL_DIR` usando `safe_serialization` para compatibilidad con `safetensors`.


In [ ]:
# MERGE LoRA a MODELO COMPLETO
import os, gc, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Limpieza básica de memoria
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Paths
BASE_MODEL_ID    = "mistralai/Mistral-7B-Instruct-v0.3"
ADAPTER_PATH     = "./Mistral-7B-Python-Expert-v1-final"     # carpeta de trainer.save_model(...)
MERGED_MODEL_DIR = "./Code-Specialist-7b"    # salida del modelo fusionado

print("Base:", BASE_MODEL_ID)
print("Adapter:", ADAPTER_PATH)
print("Salida:", MERGED_MODEL_DIR)

Base: mistralai/Mistral-7B-Instruct-v0.3
Adapter: ./Mistral-7B-Python-Expert-v1-final
Salida: ./Code-Specialist-7b


In [ ]:
# Cargar base en CPU
DTYPE_CPU = torch.bfloat16   
print("Cargando base en CPU...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=DTYPE_CPU,
    device_map=None,              # CPU
    trust_remote_code=True
)

print("Montando adaptadores LoRA...")
peft_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
    is_trainable=False
)

print("Fusionando (merge_and_unload)...")
merged_model = peft_model.merge_and_unload()
print("Merge completo.")

`torch_dtype` is deprecated! Use `dtype` instead!


Cargando base en CPU...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Montando adaptadores LoRA...
Fusionando (merge_and_unload)...
Merge completo.


In [ ]:
# Asegurar carpeta de salida
os.makedirs(MERGED_MODEL_DIR, exist_ok=True)

# Guardado en safetensors (recomendado)
print("Guardando pesos fusionados...")
merged_model.save_pretrained(MERGED_MODEL_DIR, safe_serialization=True)

# Guardar tokenizer con pad_token configurado
print("Guardando tokenizer...")
tok = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.save_pretrained(MERGED_MODEL_DIR)

try:
    merged_model.config.torch_dtype = str(merged_model.dtype).replace("torch.", "")
    merged_model.config.save_pretrained(MERGED_MODEL_DIR)
except Exception as e:
    print("Aviso: no se pudo persistir torch_dtype en config:", e)

print("Guardado listo en:", MERGED_MODEL_DIR)

Guardando pesos fusionados...
Guardando tokenizer...
Guardado listo en: ./Code-Specialist-7b


#### Autor
**Ricardo Urdaneta**